In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/duygu-analizi-icin-urun-yorumlari/magaza_yorumlari_duygu_analizi.csv


In [2]:
data = pd.read_csv('/kaggle/input/duygu-analizi-icin-urun-yorumlari/magaza_yorumlari_duygu_analizi.csv',encoding="utf-16")

In [3]:
data20 = data.head(20)

In [4]:
def preprocess_text(text):
    # Küçük harflere dönüştürme
    text = text.lower()
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    # Stop words'leri kaldırma
    stop_words = set(stopwords.words('turkish'))  # Türkçe stop words'leri yükleme
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# 'Görüş' sütunundaki cümleleri bir liste olarak almak
cümleler = data20["Görüş"]

# Veri setindeki cümleleri işleme ve temizleme
cleaned_cümleler = [preprocess_text(cümle) for cümle in cümleler]

# Temizlenmiş cümleleri gösterme
for i, cümle in enumerate(cleaned_cümleler):
    print(f"Cümle {i+1}: {cümle}")

Cümle 1: ses kalitesi ergonomisi rezalet sony olduğu aldım 4de 1 fiyatına çin replika ürün alsaydım iyiydi kesinlikle tavsiye etmiyorum
Cümle 2: hizli teslimat tesekkürler
Cümle 3: ses olayı süpergece çalıştır sıkıntı yokkablo uzun işinizi rahat ettirirçekme olayı son derece güzel içiniz rahat olsundiğerlerini saymıyorum bile
Cümle 4: geldi bigün kullandık hemen bozoldu hiçtavsiye etmem
Cümle 5: kulaklığın sesi kaliteli falan değil aleti öve öve bitiremeyen yorumlar şüpheli tizler yok olan boğuk çıkıyor bas kaba saba ben buradayım diyor kalite yok iyi ses arayanlara tavsiye etmem hayatımda aldığım ilk snopy marka üründü onu yorumlara güvenerek aldım pişman oldum hepsiburadanın sahte yorumlara karşı önlem alması gerekiyor artık
Cümle 6: giriş seviyesindeki kullanıcılar kabul edilebilir sonuçlar veren bir lens fokus motoru makro çekimleri kaliteli sonuçlar veriyor şunu belirtmek lazım belli bir süre kullandıktan sonra kaliteli bir lens ihtiyacı hissedeceksiniz
Cümle 7: kullanisli baya
Cü

In [5]:
# Kelimelere ayırma ve stop words'leri kaldırma
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in cleaned_cümleler]
stop_words = set(stopwords.words('turkish'))  # Türkçe stop words'leri yükleme
tokenized_sentences = [[word for word in sentence if word not in stop_words] for sentence in tokenized_sentences]

# Word2Vec modeli eğitme
model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1)

# Modeli kaydetme
model.save("word2vec_model.bin")

In [6]:
# Kelimelere ayırma ve stop words'leri kaldırma
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in cleaned_cümleler]
stop_words = set(stopwords.words('turkish'))  # Türkçe stop words'leri yükleme
tokenized_sentences = [[word for word in sentence if word not in stop_words] for sentence in tokenized_sentences]

# Cümle vektörlerini oluşturma
sentence_vectors = []
for tokenized_sentence in tokenized_sentences:
    vectors = []
    for word in tokenized_sentence:
        if word in model.wv:
            vectors.append(model.wv[word])  # Kelimenin vektörünü al
    if vectors:  # Eğer cümlede en az bir kelimenin vektörü varsa
        sentence_vector = sum(vectors) / len(vectors)  # Cümle vektörünü hesapla (kelime vektörlerinin ortalaması)
        sentence_vectors.append(sentence_vector)
    else:
        sentence_vectors.append(None)  # Cümledeki hiçbir kelime modelde bulunamazsa None ekle

# Oluşturulan cümle vektörlerini gösterme
for i, vector in enumerate(sentence_vectors):
    print(f"Cümle {i+1} vektörü: {vector}")

Cümle 1 vektörü: [ 1.0359008e-03 -7.0758938e-04 -1.6831311e-03 -5.0987996e-04
  1.3510631e-03  5.8232871e-04  1.8527458e-04  2.3893225e-03
  1.9002953e-03  3.1670861e-04  1.5389180e-03  1.3694548e-04
 -8.9521694e-04  1.2770434e-03  2.1011615e-03  1.9397715e-03
 -4.2822605e-04  2.0621640e-04  1.2261214e-04 -1.0923891e-03
 -1.0156099e-03  5.4836541e-04  2.8531218e-04  2.1100625e-04
  1.4991789e-03 -1.4676407e-03  2.7976551e-03 -1.1091265e-04
 -6.5229007e-04  1.0132397e-03  2.6181152e-03 -1.8308003e-03
 -1.8122451e-03  1.4078243e-03  1.5635719e-04  1.6779228e-03
  1.8962601e-03  1.0297692e-03 -4.8602984e-04 -1.5265878e-03
 -8.0577721e-04  2.1923509e-04  1.2708610e-03 -7.4332306e-04
  8.0114300e-04 -1.4412066e-03 -2.0793761e-04  1.9721538e-03
 -1.8182413e-03  2.7067732e-04 -4.9810007e-04 -6.6344091e-04
 -9.3652378e-04  8.8221021e-04 -7.2905771e-04  1.5859916e-03
  7.8101299e-04  1.1670188e-03 -1.3350209e-04  1.2931642e-03
  6.0019240e-04  9.9002058e-04  2.0510156e-03 -1.0339604e-03
 -1.368